In [ ]:
import time

import numpy as np
import pandas as pd
import requests

In [ ]:
with requests.get('https://opensky-network.org/api/states/all?extended=1') as response:
    response.raise_for_status()
    data = response.json()
print(time.time(), data['time'])

In [ ]:
columns = {
    'icao24': str,
    'callsign': str,
    'origin_country': str,
    'time_position': float,  # float to accommodate nulls
    'last_contact': int,
    'longitude': float,
    'latitude': float,
    'baro_altitude': float,
    'on_ground': bool,
    'velocity': float,
    'true_track': float,
    'vertical_rate': float,
    'sensors': object,
    'geo_altitude': float,
    'squawk': str,
    'spi': bool,
    'position_source': int,
    'category': int,
}

In [ ]:
df = pd.DataFrame(data['states'], columns=columns.keys()).astype(columns)
df

In [ ]:
df[df.latitude.isna() | df.longitude.isna()]

In [ ]:
df[df.on_ground]

In [ ]:
df[df.squawk != 'None']

In [ ]:
abs(df.baro_altitude - df.geo_altitude) / np.minimum(df.geo_altitude, df.baro_altitude)

In [ ]:
last_contact_diff = data['time'] - df.last_contact
np.min(last_contact_diff), np.median(last_contact_diff), np.mean(last_contact_diff), np.max(last_contact_diff)

In [ ]:
time_position = np.array([state[3] for state in data['states'] if state[3] is not None])
time_position_diff = data['time'] - time_position
np.min(time_position_diff), np.median(time_position_diff), np.mean(time_position_diff), np.max(time_position_diff)

In [ ]:
sum(last_contact_diff < 60) / len(last_contact_diff), sum(time_position_diff < 60) / len(time_position_diff)

In [ ]:
df[df.time_position > df.last_contact]

In [ ]:
df[data['time'] < df.time_position]

In [ ]:
df[data['time'] < df.last_contact]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

In [ ]:
sns.histplot(last_contact_diff, log_scale=True)
plt.show()

In [ ]:
sns.histplot(time_position_diff, log_scale=True)
plt.show()

In [ ]:
df.time_position.argmin()

In [ ]:
df.icao24[df.time_position.argmin()]

In [ ]:
df[df.time_position.argmin():df.time_position.argmin()+1]

In [ ]:
lag_df = df.copy()
lag_df['position_lag'] = data['time'] - df.time_position
lag_df['contact_lag'] = data['time'] - df.last_contact

In [ ]:
lag_df = lag_df.sort_values('position_lag', ascending=False)
lag_df

In [ ]:
lag_df = lag_df.sort_values('contact_lag', ascending=False)
lag_df

In [ ]:
df.describe()